## Preparación de datos

### Importación de Librerías

In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import time
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb
import gc

### Carga y Preprocesamiento de Datos

In [30]:
# Carga los datos
data = pd.read_csv('datasets/car_data.csv')

In [31]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [32]:
#Información general sobre el dataset
print(data.info())
print()
# Estadísticas descriptivas de las variables numéricas
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [33]:
# Eliminando columnas irrelevantes
data = data.drop(columns=['DateCrawled', 'DateCreated', 'NumberOfPictures', 'LastSeen', 'PostalCode'])

Eliminación de Columnas Irrelevantes: Se eliminan columnas que no aportan información relevante para el modelo.

In [34]:
# Porcentaje de valores nulos en cada columna
null_percent = data.isnull().mean() * 100
print(null_percent)

Price                 0.000000
VehicleType          10.579368
RegistrationYear      0.000000
Gearbox               5.596709
Power                 0.000000
Model                 5.560588
Mileage               0.000000
RegistrationMonth     0.000000
FuelType              9.282697
Brand                 0.000000
NotRepaired          20.079070
dtype: float64


Cálculo de Valores Nulos: Se calcula el porcentaje de valores nulos en cada columna para decidir cómo manejarlos.

In [35]:
# Imputación de valores nulos
data['VehicleType'].fillna('Desconocido', inplace=True)
data['Gearbox'].fillna(data['Gearbox'].mode()[0], inplace=True)
data['Model'].fillna('Desconocido', inplace=True)
data['FuelType'].fillna('Desconocido', inplace=True)
data['NotRepaired'].fillna(data['NotRepaired'].mode()[0], inplace=True)

C:\Users\juan6\AppData\Local\Temp\ipykernel_16016\1599145848.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['VehicleType'].fillna('Desconocido', inplace=True)
C:\Users\juan6\AppData\Local\Temp\ipykernel_16016\1599145848.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



Imputación de Valores Nulos: Se reemplazan los valores nulos con valores predeterminados, como 'Desconocido' o la moda de la columna.

In [36]:
# Verificar que no haya valores nulos restantes
print(data.isnull().sum())

Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
dtype: int64


In [37]:
# Filtrar outliers basado en un rango razonable para características numéricas
data = data[(data['RegistrationYear'] >= 1900) & (data['RegistrationYear'] <= 2024)] # eliminamos años muy antiguos y fechas fututras a la actual
data = data[(data['Power'] > 50) & (data['Power'] < 2000)] # investigando este podria ser el rango
data = data[data['Price'] > 100]  # Eliminando precios negativos o demasiado bajos

# Revisar los datos después de la limpieza
print(data.describe())

               Price  RegistrationYear          Power        Mileage  \
count  295312.000000     295312.000000  295312.000000  295312.000000   
mean     4943.692407       2003.528939     123.808809  128572.729859   
std      4603.262675          6.685565      62.523248   36526.152516   
min       101.000000       1910.000000      51.000000    5000.000000   
25%      1450.000000       1999.000000      82.000000  125000.000000   
50%      3300.000000       2004.000000     115.000000  150000.000000   
75%      7000.000000       2008.000000     150.000000  150000.000000   
max     20000.000000       2019.000000    1999.000000  150000.000000   

       RegistrationMonth  
count      295312.000000  
mean            5.987667  
std             3.589068  
min             0.000000  
25%             3.000000  
50%             6.000000  
75%             9.000000  
max            12.000000  


Removemos y corregir valores atípicos, que son errores de entrada (por ejemplo, RegistrationYear igual a 9999).

Establecer rangos razonables para las características basados en el conocimiento del dominio (por ejemplo, el año de registro no debería ser mayor al año actual).

In [38]:
# Codificación One-Hot Encoding de las variables categóricas
data_ohe = pd.get_dummies(data, columns=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'], drop_first=True)


In [39]:
for col in ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']:
    print(f"{col} tiene {data[col].nunique()} valores únicos") # revisar valores unicos

VehicleType tiene 9 valores únicos
Gearbox tiene 2 valores únicos
Model tiene 249 valores únicos
FuelType tiene 8 valores únicos
Brand tiene 40 valores únicos
NotRepaired tiene 2 valores únicos


Revise los valores unicos, con ello decidi agrupar los valores menos comunes para evitar la alta cardinalidad.

In [40]:
# Definir el umbral mínimo de frecuencia para mantener las categorías tal cual
threshold_model = 5000  
threshold_brand = 5000  

# Agrupar categorías menos frecuentes en 'Model'
model_value_counts = data['Model'].value_counts()
models_to_replace = model_value_counts[model_value_counts < threshold_model].index
data['Model'] = data['Model'].replace(models_to_replace, 'Otro')

# Agrupar categorías menos frecuentes en 'Brand'
brand_value_counts = data['Brand'].value_counts()
brands_to_replace = brand_value_counts[brand_value_counts < threshold_brand].index
data['Brand'] = data['Brand'].replace(brands_to_replace, 'Otro')


In [41]:
for col in ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']:
    print(f"{col} tiene {data[col].nunique()} valores únicos") # nuevamente reviso valores unicos

VehicleType tiene 9 valores únicos
Gearbox tiene 2 valores únicos
Model tiene 16 valores únicos
FuelType tiene 8 valores únicos
Brand tiene 12 valores únicos
NotRepaired tiene 2 valores únicos


In [42]:
data_ohe = pd.get_dummies(data, columns=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'], drop_first=True)

Luego de reducir los valores unicos agrupando se aplica nuevamente One_Hot encoding

Codificación One-Hot Encoding: Se convierte cada categoría en una columna binaria, lo que es necesario para los modelos que no pueden manejar variables categóricas directamente.

In [43]:
data_ohe.head()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,VehicleType_bus,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,...,Brand_fiat,Brand_ford,Brand_mercedes_benz,Brand_opel,Brand_peugeot,Brand_renault,Brand_seat,Brand_skoda,Brand_volkswagen,NotRepaired_yes
1,18300,2011,190,125000,5,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
2,9800,2004,163,125000,8,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1500,2001,75,150000,6,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,3600,2008,69,90000,7,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
5,650,1995,102,150000,10,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True


In [44]:
# Definir características 
features = data_ohe.drop(columns=['Price'], axis=1)
target = data_ohe['Price']

# División de los datos en entrenamiento, validación y prueba
features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size=0.4, random_state=12345)

# Dividir el conjunto 'temp' en validación y prueba
features_valid, features_test, target_valid, target_test = train_test_split(features_temp, target_temp, test_size=0.5, random_state=12345)

División de Datos: Se dividen los datos en conjuntos de entrenamiento, validación y prueba.

In [45]:
# Características numéricas a estandarizar
numeric = ['RegistrationYear', 'Power', 'Mileage', 'RegistrationMonth']

# Crear y ajustar el escalador
scaler = StandardScaler()
scaler.fit(features_train[numeric])
pd.options.mode.chained_assignment = None

# Transformar las características numéricas en los conjuntos de entrenamiento, validación y prueba
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

# Ahora tienes tus datos divididos en 3 conjuntos: features_train, features_valid, features_test

Estandarización de Características Numéricas: Se escalan las características numéricas para que todas tengan media 0 y desviación estándar 1, lo cual es importante para la mayoría de los modelos de machine learning.

### Codificación Ordinal para Modelos Basados en Árboles

In [46]:
# Codificación de etiquetas para árboles de decisión y bosques aleatorios
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(data), columns=data.columns)

# Separar características y objetivo
target_tree = data_ordinal['Price']
features_tree = data_ordinal.drop(['Price'], axis=1)

# Dividir los datos en entrenamiento, validación y prueba
features_tree_train, features_tree_temp, target_tree_train, target_tree_temp = train_test_split(features_tree, target_tree, test_size=0.4, random_state=12345)

# Dividir el conjunto 'temp' en validación y prueba
features_tree_valid, features_tree_test, target_tree_valid, target_tree_test = train_test_split(features_tree_temp, target_tree_temp, test_size=0.5, random_state=12345)

# Ahora tienes features_tree_train, features_tree_valid y features_tree_test

Codificación Ordinal: Para modelos basados en árboles, se usa la codificación ordinal que asigna valores enteros a las categorías, ya que estos modelos pueden manejar variables categóricas directamente.


División de Datos: Se realiza nuevamente la división en conjuntos de entrenamiento, validación y prueba para los datos codificados de forma ordinal.

In [47]:
del data
del data_ohe

En este proyecto se presento problemas con la memoria, por lo que se decidio eliminar valores no necesarios

## Entrenamiento del modelo 

### Regresión Lineal:


In [48]:
# Entrenamiento y Validación del Modelo

# Entrenamiento y evaluación en el conjunto de validación
start_train_time = time.time()
model_lr = LinearRegression()
model_lr.fit(features_train, target_train)
end_train_time = time.time()
train_time_lr = end_train_time - start_train_time

start_pred_time = time.time()
predictions_valid = model_lr.predict(features_valid)
end_pred_time = time.time()
pred_time_lr = end_pred_time - start_pred_time

rmse_lr_valid = mean_squared_error(target_valid, predictions_valid)**0.5
print(f'Regresión Lineal - RMSE en validación: {rmse_lr_valid:.2f}, Tiempo de entrenamiento: {train_time_lr:.2f} s, Tiempo de predicción: {pred_time_lr:.2f} s')

# Evaluación final en el conjunto de prueba
predictions_test = model_lr.predict(features_test)
rmse_lr_test = mean_squared_error(target_test, predictions_test)**0.5
print(f'Regresión Lineal - RMSE en prueba: {rmse_lr_test:.2f}')

del model_lr, predictions_valid, predictions_test

Regresión Lineal - RMSE en validación: 2877.20, Tiempo de entrenamiento: 0.33 s, Tiempo de predicción: 0.03 s
Regresión Lineal - RMSE en prueba: 2859.03


Entrenamiento: Se entrena un modelo de regresión lineal con los datos estandarizados.


Evaluación: Se calcula el RMSE para evaluar la precisión del modelo con un resultado de 2877.20 para el conjunto de validación y 2859.03 para el conjunto de prueba.

### Árbol de Decisión:

In [49]:
# Árbol de Decisión
best_rmse_dt = float("inf")
best_depth_dt = 0

for depth in range(1, 11):
    start_train_time = time.time()
    model_dt = DecisionTreeRegressor(max_depth=depth, random_state=485)
    model_dt.fit(features_tree_train, target_tree_train)
    end_train_time = time.time()
    train_time_dt = end_train_time - start_train_time
    
    start_pred_time = time.time()
    predictions_valid = model_dt.predict(features_tree_valid)
    end_pred_time = time.time()
    pred_time_dt = end_pred_time - start_pred_time

    rmse = mean_squared_error(target_tree_valid, predictions_valid)**0.5
    if rmse < best_rmse_dt:
        best_rmse_dt = rmse
        best_depth_dt = depth
        best_train_time_dt = train_time_dt
        best_pred_time_dt = pred_time_dt

    print(f'Decision Tree max_depth={depth} RMSE en validación: {rmse:.2f}, Tiempo de entrenamiento: {train_time_dt:.2f} s, Tiempo de predicción: {pred_time_dt:.2f} s')

# Evaluación final del mejor Árbol de Decisión en el conjunto de prueba
model_dt = DecisionTreeRegressor(max_depth=best_depth_dt, random_state=485)
model_dt.fit(features_tree_train, target_tree_train)
predictions_test_dt = model_dt.predict(features_tree_test)
rmse_dt_test = mean_squared_error(target_tree_test, predictions_test_dt)**0.5
print(f'Decision Tree - RMSE en prueba: {rmse_dt_test:.2f}')

del model_dt, predictions_valid, predictions_test_dt

Decision Tree max_depth=1 RMSE en validación: 732.60, Tiempo de entrenamiento: 0.05 s, Tiempo de predicción: 0.01 s
Decision Tree max_depth=2 RMSE en validación: 642.42, Tiempo de entrenamiento: 0.07 s, Tiempo de predicción: 0.00 s
Decision Tree max_depth=3 RMSE en validación: 586.64, Tiempo de entrenamiento: 0.11 s, Tiempo de predicción: 0.00 s
Decision Tree max_depth=4 RMSE en validación: 532.01, Tiempo de entrenamiento: 0.14 s, Tiempo de predicción: 0.00 s
Decision Tree max_depth=5 RMSE en validación: 490.77, Tiempo de entrenamiento: 0.16 s, Tiempo de predicción: 0.00 s
Decision Tree max_depth=6 RMSE en validación: 458.04, Tiempo de entrenamiento: 0.19 s, Tiempo de predicción: 0.01 s
Decision Tree max_depth=7 RMSE en validación: 434.61, Tiempo de entrenamiento: 0.21 s, Tiempo de predicción: 0.01 s
Decision Tree max_depth=8 RMSE en validación: 416.59, Tiempo de entrenamiento: 0.23 s, Tiempo de predicción: 0.01 s
Decision Tree max_depth=9 RMSE en validación: 403.80, Tiempo de entrenam

Entrenamiento y Evaluación con Diferentes Profundidades: Se entrena un árbol de decisión con diferentes profundidades y se evalúa cada uno usando RMSE, y con la mejor profundidad que es 10 se evalua con un RMSE del conjunto de prueba de 393.28.

### Bosque Aleatorio:

In [50]:
# Bosque Aleatorio
best_rmse_rf = float("inf")
best_depth_rf = 0

for depth in range(1, 11):
    start_train_time = time.time()
    model_rf = RandomForestRegressor(n_estimators=100, max_depth=depth, random_state=485)
    model_rf.fit(features_tree_train, target_tree_train)
    end_train_time = time.time()
    train_time_rf = end_train_time - start_train_time

    start_pred_time = time.time()
    predictions_valid = model_rf.predict(features_tree_valid)
    end_pred_time = time.time()
    pred_time_rf = end_pred_time - start_pred_time

    rmse = mean_squared_error(target_tree_valid, predictions_valid)**0.5
    if rmse < best_rmse_rf:
        best_rmse_rf = rmse
        best_depth_rf = depth
        best_train_time_rf = train_time_rf
        best_pred_time_rf = pred_time_rf

    print(f'Random Forest max_depth={depth} RMSE en validación: {rmse:.2f}, Tiempo de entrenamiento: {train_time_rf:.2f} s, Tiempo de predicción: {pred_time_rf:.2f} s')

# Evaluación final del mejor Bosque Aleatorio en el conjunto de prueba
model_rf = RandomForestRegressor(n_estimators=100, max_depth=best_depth_rf, random_state=485)
model_rf.fit(features_tree_train, target_tree_train)
predictions_test_rf = model_rf.predict(features_tree_test)
rmse_rf_test = mean_squared_error(target_tree_test, predictions_test_rf)**0.5
print(f'Random Forest - RMSE en prueba: {rmse_rf_test:.2f}')

del model_rf, predictions_valid, predictions_test_rf

gc.collect()


Random Forest max_depth=1 RMSE en validación: 732.60, Tiempo de entrenamiento: 3.01 s, Tiempo de predicción: 0.07 s
Random Forest max_depth=2 RMSE en validación: 642.07, Tiempo de entrenamiento: 5.29 s, Tiempo de predicción: 0.11 s
Random Forest max_depth=3 RMSE en validación: 583.20, Tiempo de entrenamiento: 7.33 s, Tiempo de predicción: 0.14 s
Random Forest max_depth=4 RMSE en validación: 526.47, Tiempo de entrenamiento: 9.14 s, Tiempo de predicción: 0.18 s
Random Forest max_depth=5 RMSE en validación: 481.60, Tiempo de entrenamiento: 11.92 s, Tiempo de predicción: 0.24 s
Random Forest max_depth=6 RMSE en validación: 448.65, Tiempo de entrenamiento: 13.03 s, Tiempo de predicción: 0.25 s
Random Forest max_depth=7 RMSE en validación: 423.65, Tiempo de entrenamiento: 14.35 s, Tiempo de predicción: 0.29 s
Random Forest max_depth=8 RMSE en validación: 404.77, Tiempo de entrenamiento: 15.96 s, Tiempo de predicción: 0.35 s
Random Forest max_depth=9 RMSE en validación: 389.48, Tiempo de entr

352

Entrenamiento y Evaluación con Diferentes Profundidades: Se entrena un bosque aleatorio con diferentes profundidades de árboles y se evalúa cada modelo usando RMSE, y con la mejor profundidad que es 10 se evalua con un RMSE del conjunto de prueba de 375.93.

### LightGBM:

In [51]:
# LightGBM

train_data = lgb.Dataset(features_tree_train, label=target_tree_train)
valid_data = lgb.Dataset(features_tree_valid, label=target_tree_valid, reference=train_data)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': 10,
    'num_leaves': 1025,  
    'learning_rate': 0.1,
    'verbose': -1  
}

start_train_time = time.time()
model_lgb = lgb.train(params, train_data, valid_sets=[valid_data], num_boost_round=100)
end_train_time = time.time()
train_time_lgb = end_train_time - start_train_time

start_pred_time = time.time()
predictions_valid_lgb = model_lgb.predict(features_tree_valid)
end_pred_time = time.time()
pred_time_lgb = end_pred_time - start_pred_time

rmse_lgb_valid = mean_squared_error(target_tree_valid, predictions_valid_lgb) ** 0.5
print(f'LightGBM - RMSE en validación: {rmse_lgb_valid:.2f}, Tiempo de entrenamiento: {train_time_lgb:.2f} s, Tiempo de predicción: {pred_time_lgb:.2f} s')

# Evaluación final en el conjunto de prueba
predictions_test_lgb = model_lgb.predict(features_tree_test)
rmse_lgb_test = mean_squared_error(target_tree_test, predictions_test_lgb) ** 0.5
print(f'LightGBM - RMSE en prueba: {rmse_lgb_test:.2f}')

# Liberar memoria
del predictions_valid_lgb, predictions_test_lgb, train_data, valid_data
gc.collect()

LightGBM - RMSE en validación: 316.31, Tiempo de entrenamiento: 2.57 s, Tiempo de predicción: 0.38 s
LightGBM - RMSE en prueba: 315.84


42

Preparación de Datos: Los datos se convierten en un formato compatible con LightGBM.

Entrenamiento: Se entrena el modelo con los parámetros definidos y se mide el tiempo de entrenamiento.

Evaluación: Se calculan las predicciones, el RMSE y se mide el tiempo de predicción.

Finalmente calculamos el RMSE de prueba dandonos un 315.84.

## Analisis del modelo

- Regresión Lineal: Mostró un RMSE de 2859.03 para el conjunto de prueba, lo que indica que no captura bien la complejidad de los datos.

- Árbol de Decisión: Mejoró significativamente el RMSE a medida que aumentaba la profundidad, con un mejor resultado de 461.91 en max_depth=10.

- Bosque Aleatorio: Superó al árbol de decisión, alcanzando un RMSE de 442.45 en max_depth=10, mostrando la ventaja de agregar múltiples árboles y reducir el sobreajuste.

In [52]:
# Medir el tiempo de predicción en el conjunto de prueba
start_pred_time = time.time()
predictions_test_lgb = model_lgb.predict(features_tree_test)
end_pred_time = time.time()
pred_time_lgb_test = end_pred_time - start_pred_time

# Calcular RMSE en el conjunto de prueba
rmse_lgb_test = mean_squared_error(target_tree_test, predictions_test_lgb) ** 0.5
print(f'LightGBM - RMSE en prueba: {rmse_lgb_test:.2f}, Tiempo de predicción en prueba: {pred_time_lgb_test:.2f} s')

# Liberar memoria
del predictions_test_lgb
gc.collect()

LightGBM - RMSE en prueba: 315.84, Tiempo de predicción en prueba: 0.21 s


4

Después de entrenar y validar varios modelos, incluyendo Regresión Lineal, Árbol de Decisión y Bosque Aleatorio, seleccionamos LightGBM como el mejor modelo para este problema debido a su bajo RMSE en el conjunto de validación (316.31) y su rápido tiempo de predicción.

Para confirmar la robustez de este modelo, se realizó una evaluación final utilizando el conjunto de prueba.

El modelo LightGBM arrojó los siguientes resultados en el conjunto de prueba:

- RMSE en prueba: 315.84
- Tiempo de predicción en prueba: 0.21 segundos

Estos resultados son consistentes con el rendimiento observado en la validación, lo que indica que el modelo generaliza bien a nuevos datos.

Con estos resultados, LightGBM demuestra ser el modelo más efectivo y confiable para predecir el valor de mercado de los coches de segunda mano en este proyecto.

## Conclusiones:

- LightGBM demostró ser el modelo con la mejor calidad de predicción, con un RMSE de 315.84 para el conjunto de prueba, lo que sugiere que este modelo es el más preciso para predecir el precio de los coches de segunda mano.

- Bosque Aleatorio también mostró un buen rendimiento, con un RMSE que mejora con la profundidad del modelo, alcanzando 375.93 a una profundidad de 10 para el conjunto de prueba.

- Árbol de Decisión proporcionó una solución más simple pero menos precisa, con un RMSE de 393.28 para conjunto de prueba.

- Regresión Lineal tuvo el RMSE más alto (2859.03), confirmando que las relaciones en los datos no son lineales y requieren modelos más complejos para capturar patrones precisos.

- Bosque Aleatorio demostró ser efectivo, pero a un costo de tiempo de entrenamiento significativamente más largo, especialmente a medida que aumenta la profundidad del modelo.

- LightGBM se destaca como la mejor opción cuando se busca un equilibrio entre precisión y eficiencia de tiempo, tanto en entrenamiento como en predicción.

- LightGBM es la opción más adecuada para la implementación en producción, ya que ofrece la mejor calidad de predicción con tiempos de entrenamiento y predicción razonables.

- El preprocesamiento y la codificación de variables categóricas fueron cruciales para el rendimiento de los modelos. La elección adecuada de técnicas de preprocesamiento, como la imputación de valores nulos y la estandarización, mejoró el rendimiento de los modelos.

- Se recomienda utilizar LightGBM como el modelo principal para la predicción del valor de mercado de los coches, debido a su equilibrio entre precisión y eficiencia de tiempo. 
